In [11]:
!pip install selenium --upgrade
!pip install webdriver_manager --upgrade

In [71]:
# S&P 500 companies list

import requests
from bs4 import BeautifulSoup

URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response = requests.get(URL)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the list of S&P 500 companies
table = soup.find('table', {'class': 'wikitable sortable'})

# Extract company names from the table
companies_500 = []
for row in table.findAll('tr')[1:]:
    columns = row.findAll('td')
    company_name = columns[0].get_text(strip=True)
    companies_500.append(company_name)

print(companies_500)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AES', 'AFL', 'A', 'ABNB', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BG', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'C

In [72]:
first_ten_companies = companies_500[:10]
print(first_ten_companies)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AES']


In [73]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time



In [75]:
# Ensure chromedriver is set up
ChromeDriverManager().install()

# Set Chrome to automatically download PDF files
prefs = {
    "download.default_directory": "/Users/neuefische/Downloads/test_ESG", # this directory needs to be adjusted accordingly
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
}

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('prefs', prefs)

# Initialize the browser
driver = webdriver.Chrome(options=chrome_options)

company_list = companies_500[:10] # List of S&P 500 companies

wait = WebDriverWait(driver, 1)

for company in company_list:
    try:
        # Step 1: Navigate to the website
        driver.get("https://www.responsibilityreports.com/")

        # Step 2: Interact with the search bar
        search_bar = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > div.container > section.banner_section > div > div > div.left_section > form > input[type=text]:nth-child(2)")))
        search_bar.send_keys(company)
        search_bar.submit()

        # Step 3: Select the company from search results
        company_link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > div.container > section.category_section > div.apparel_stores_company_list > ul > li:nth-child(2) > span.companyName > a")))
        company_link.click()

        # Step 4: Download the most recent report
        try:
            most_recent_link = driver.find_element(By.CSS_SELECTOR, "div.most_recent_content_block a")
            most_recent_link.click()

            # Close the pop-up if present
            try:
                close_popup = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "body > div.container > div > div > div.close_popup > a")))
                close_popup.click()
            except:
                pass  # If popup is not present, continue to the next step

        except Exception as e:
            print(f"Unable to download the most recent report for {company}. Error: {e}")


        # Step 5: Download archived reports
        archived_links = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.archived_report_content_block a")))

        clicked_links = set()  # This set will store URLs of already clicked reports

        # Click on each archived report link
        for link in archived_links:
            link_url = link.get_attribute('href')
            if link_url not in clicked_links:
                ActionChains(driver).move_to_element(link).click(link).perform()
                clicked_links.add(link_url)  # Add the clicked link to the set
                time.sleep(1)  # wait for 1 seconds to allow the download to initiate

    except Exception as e:
        print(f"Error with company {company}: {e}")
        continue


Unable to download the most recent report for MMM. Error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.most_recent_content_block a"}
  (Session info: chrome=116.0.5845.187); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010055e65c chromedriver + 4318812
1   chromedriver                        0x0000000100556d00 chromedriver + 4287744
2   chromedriver                        0x00000001001887ec chromedriver + 296940
3   chromedriver                        0x00000001001c6048 chromedriver + 548936
4   chromedriver                        0x00000001001fed28 chromedriver + 781608
5   chromedriver                        0x00000001001ba178 chromedriver + 500088
6   chromedriver                        0x00000001001bafc0 chromedriver + 503744
7   chromedriver                        0x0000000100

In [47]:
# exit driver

driver.quit()